In [1305]:
import pandas as pd
import numpy as np
import random

import seaborn as sns
import matplotlib.pyplot as plt

from solver import Instance, Solver
from argparse import Namespace
from gurobipy import Model, GRB, tupledict
import json

import ast

In [1306]:
city = "berlin"
demand_baseline = "1.00"
demand_type = "uniform"

demand_file = f'{city}_db={demand_baseline}_dt={demand_type}.csv'
print(demand_file)

berlin_db=1.00_dt=uniform.csv


In [1307]:
demand_df = pd.read_csv(demand_file, index_col=0)

In [1308]:
# PARAMETERS
# Regions
R = sorted(list(demand_df['region_id'].unique()))

region_area_map = {}
for r in R:
    for a in list(demand_df.query(f'region_id == {r}')['area_id'].unique()):
        region_area_map[a] = r
        
# Areas 
A = {}
area_map = {a: i for i, a in enumerate(list(demand_df['area_id'].unique())) }
areas = list(area_map.keys())
n_areas = len(areas)
for r in R:
    A[r] = [ area_map[a] for a in list(demand_df.query(f'region_id == {r}')['area_id'].unique()) ]

# Employees
n_employees = 12
employees = [e for e in range(n_employees)]
E = {}
#for r in R:
#    E[r] = employees
E[0] = [0,1,2]
E[1] = [3,4,5]
E[2] = [6,7,8]
E[3] = [9,10,11]
 
# Set of all shifts available
P = {}
n_shifts = 2
shifts = [s for s in range(n_shifts)]
for r in R:
    P[r] = shifts

# WE NEED TO SET starting and end time for each shift!! and ADD THE CONSTRAINTS so no one can work before or after this time!
shifts_start = {0: 0, 1:2}
shifts_end = {0: 4, 1:6}

# Periods
n_periods = 8
Theta = [i for i in range(n_periods)]

# Days
n_days = 7
D = [d for d in range(n_days)]

# Set of demand scenarios
# TODO

# Hours in shift
h = {}
n_hours = 8
for p in range(n_shifts):
    h[p] = n_hours

# Cost outsource
c_out = 3

# Number of deliveries to perform (n_{a theta d})
deliveries = np.zeros((n_areas, n_periods, n_days))
for i, a in enumerate(areas):
    for k, d in enumerate(D):
        period_demands = demand_df.query(f'area_id == {a} & day == {d}')
        deliveries[i, :, k] = ast.literal_eval(period_demands['demand'].values[0])

# Number of couriers needed (m_{a theta d})
couriers_needed = np.zeros((n_areas, n_periods, n_days))
for i, a in enumerate(areas):
    for k, d in enumerate(D):
        period_demands = demand_df.query(f'area_id == {a} & day == {d}')
        couriers_needed[i, :, k] = ast.literal_eval(period_demands['required_couriers'].values[0])

# Cost of employed courier (c_{a theta d})
cost_couriers = np.zeros((n_areas, n_periods, n_days))
cost_courier = 1
for i, a in enumerate(areas):
    for j, t in enumerate(Theta):
        for k, d in enumerate(D):
            cost_couriers[i, j, k] = cost_courier

# Min hours worked for employee e
min_hours_worked = 8
h_min = {e: min_hours_worked for e in employees}

# Max hours worked for employee e
max_hours_worked = 8*6
h_max = {e: max_hours_worked for e in employees}

# Max differing starts
max_unique_starts = 3
b_max = {e: max_unique_starts for e in employees}

# mega_value
M = 999_999

# DECISION VARIABLES

In [1309]:
m = Model()

# r_{e p d}
r_var = m.addVars(n_employees, n_shifts, n_days, vtype=GRB.BINARY, name='r')

# k_{e a theta d}
k_var = m.addVars(n_employees, n_areas, n_periods, n_days, vtype=GRB.BINARY, name='k')

# U_{e p}
u_var = m.addVars(n_employees, n_shifts, vtype=GRB.BINARY, name='u')

# omega_{a theta d}
omega_var = m.addVars(n_areas, n_periods, n_days, vtype=GRB.CONTINUOUS, lb=0, name='omega')

# CONSTRAINTS

In [1310]:
# 1. Connecting employees moving around areas to shift assignment p
if False:
    for r in R:
        for e in E[r]:
            for p in P[r]:
                for d in D:
                    Ar = A[r]
                    m.addConstr(
                        (sum([k_var[e, a, theta, d] for a in Ar for theta in Theta]) == 1/2 * h[p] * r_var[e,p,d] ),
                        name = f'moving_areas_{r}_{e}_{p}_{d}'          
                    )

# CORRECTED!
for r in R:
    for e in E[r]:
        for d in D:
            Ar, Pr = A[r], P[r]
            m.addConstr(
                (sum([k_var[e, a, theta, d] for a in Ar for theta in Theta]) == 1/2 * sum([h[p] * r_var[e,p,d] for p in Pr])),
                name = f'moving_areas_{r}_{e}_{d}'          
            )

In [1311]:
# 2. Employee can only be assigned to one area at a time 
for r in R:
    for e in E[r]:
        for theta in Theta:
            for d in D:
                Ar = A[r]
                m.addConstr((sum([k_var[e, a, theta, d] for a in Ar]) <= 1),
                    name = f'employee_{r}_{e}_{theta}_{d}'          
                )

# ADDED: They only can be assigned one region!
#for e in employees:
#    for theta in Theta:
#        for d in D:
#            m.addConstr((sum([k_var[e, area_map[a], theta, d] for a in areas]) <= 1),
#                name = f'only_one_area_{e}_{theta}_{d}'          
#            )

In [1312]:
# ADDED - Shift start and end times
for shift in shifts:
    start = shifts_start[shift]
    for theta in Theta[:start]:
        for e in employees:
            for a in areas:
                for d in D:
                    m.addConstr((k_var[e, area_map[a], theta, d] * r_var[e, shift, d] == 0),
                        name = f'start_time_{e}_{a}_{shift}_{d}'          
                    )

for shift in shifts:
    end = shifts_end[shift]
    for theta in Theta[end:]:
        for e in employees:
            for a in areas:
                for d in D:
                    m.addConstr((k_var[e, area_map[a], theta, d] * r_var[e, shift, d] == 0),
                        name = f'end_time_{e}_{a}_{shift}_{d}'          
                    )

In [1313]:
# 3. Employee can only work one shift a day
for r in R:
    for es in E[r]:
        for d in D:
            Pr = P[r]
            m.addConstr((sum([r_var[e, p, d] for p in Pr]) <= 1),
                name = f'only_one_shift_{r}_{e}_{theta}_{d}'          
            )

In [1314]:
# 4. One rest day a week
for r in R:
    for e in E[r]:
        Pr = P[r]
        m.addConstr((sum([r_var[e, p, d] for p in Pr for d in D]) <= 6),
            name = f'rest_day_{r}_{e}'          
        )

In [1315]:
# 5. Min - Max hours worked per week
for r in R:
    Pr = P[r]
    for e in E[r]:
        min_hours = h_min[e]
        max_hours = h_max[e]

        m.addConstr((sum([ h[p] * r_var[e, p, d] for p in Pr for d in D]) >= min_hours),
            name = f'min_hours_{r}_{e}'          
        )

        m.addConstr(( sum([ h[p] * r_var[e, p, d] for p in Pr for d in D]) <= max_hours),
            name = f'max_hours_{r}_{e}'          
        )

In [1316]:
# 6. Different shifting times constraint
for r in R:
    for e in E[r]:
        for p in P[r]:
            m.addConstr((sum([r_var[e, p, d] for d in D]) <= u_var[e, p] * M),
                name = f'start_times_{r}_{e}_1'          
            )

for r in R:
    for e in E[r]:
        Pr = P[r]
        m.addConstr((sum([u_var[e, p] for p in Pr]) <= b_max[e]),
            name = f'start_times_{r}_{e}_2'          
        )

In [1317]:
# 7. Employee can't work then outsource
for r in R:
    for a in A[r]:
        for theta in Theta:
            for d in D:
                # Be careful with parantheses!!
                if couriers_needed[a,theta,d] > 0:
                    factor = deliveries[a, theta, d] / couriers_needed[a, theta, d] * c_out 
                else:
                    factor = 0
                m.addConstr(
                    ((couriers_needed[a, theta, d] - sum([k_var[e, a, theta, d] for e in E[r]])) * factor <= omega_var[a, theta, d]),
                    name = f'outsource_{r}_{a}_{theta}_{d}'   
                )

# OBJECTIVE FUNCTION

In [1318]:
factor_s = 1
m.setObjective(
    sum([
        sum([cost_couriers[a, theta, d] * k_var[e, a, theta, d] for e in E[r]]) 
            + omega_var[a, theta, d]
    for r in R for a in A[r] for theta in Theta for d in D]
    )
)

In [1319]:
m.ModelSense = GRB.MINIMIZE
m.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[rosetta2])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4216 rows, 43144 columns and 32407 nonzeros
Model fingerprint: 0x50294f13
Model has 39648 quadratic constraints
Variable types: 3304 continuous, 39840 integer (39840 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Presolve removed 3355 rows and 40298 columns
Presolve time: 1.03s
Presolved: 861 rows, 2846 columns, 7234 nonzeros
Variable types: 0 continuous, 2846 integer (2842 binary)
Found heuristic solution: objective 17137.500000

Root relaxation: objective 1.642350e+04, 1542 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Ga

# SOLUTION

In [1320]:
# Total couriers in day
couriers_needed_ = []
for d in D:
    data = (
        pd.DataFrame(couriers_needed[:,:,d], index=areas, columns=Theta).reset_index()
        .rename(columns={'index': 'area'})
        #.assign(day = d)
    )

    area_day_data = (
        pd.melt(data, id_vars=['area'], value_vars=set(data.columns).difference('area'))
        .rename(columns={'variable': 'period', 'value': 'n_couriers'})
        .assign(day = d)
    )
    couriers_needed_.append(area_day_data)

couriers_needed_df = pd.concat(couriers_needed_)
couriers_needed_df.head()

,area,period,n_couriers,day
0,10115,0,1.0,0
1,10117,0,0.0,0
2,10119,0,1.0,0
3,10178,0,1.0,0
4,10179,0,1.0,0


In [1321]:
# Demand
deliveries_ = []
for d in D:
    data = (
        pd.DataFrame(deliveries[:,:,d], index=areas, columns=Theta).reset_index()
        .rename(columns={'index': 'area'})
        #.assign(day = d)
    )

    area_day_data = (
        pd.melt(data, id_vars=['area'], value_vars=set(data.columns).difference('area'))
        .rename(columns={'variable': 'period', 'value': 'deliveries'})
        .assign(day = d)
    )
    deliveries_.append(area_day_data)

deliveries_df = pd.concat(deliveries_)
deliveries_df.head(10)

,area,period,deliveries,day
0,10115,0,3.0,0
1,10117,0,0.0,0
2,10119,0,2.0,0
3,10178,0,1.0,0
4,10179,0,1.0,0
5,10243,0,5.0,0
6,10245,0,5.0,0
7,10247,0,3.0,0
8,10249,0,4.0,0
9,10315,0,2.0,0


In [1322]:
couriers_needed_df = pd.merge(couriers_needed_df, deliveries_df, on=['area', 'period', 'day'])

In [1323]:
# Employee shifts
employees_shifts = []
for e in employees:
    for p in range(n_shifts):
        for d in D:
            value = r_var[e, p, d].X
            if value > 0.5:
                hours_worked = h[p]
                employees_shifts.append([e, p, d, hours_worked])

employees_shifts_df = pd.DataFrame(employees_shifts, columns=['employee', 'shift', 'day', 'hours_worked'])
employees_shifts_df.head(10)

,employee,shift,day,hours_worked
0,0,0,5,8
1,0,1,0,8
2,0,1,2,8
3,0,1,3,8
4,0,1,4,8
5,0,1,6,8
6,1,0,0,8
7,1,0,1,8
8,1,0,4,8
9,1,1,2,8


In [1324]:
# Total hours worked
#employees_shifts_df.groupby('employee')['hours_worked'].sum()

In [1334]:
# Area Employee assignment
area_employee_assignment = []
for e in employees:
    for a in areas:
        for theta in Theta:
            for d in D:
                for p in shifts:
                    shift = int(r_var[e, p, d].X)
                value = k_var[e, area_map[a], theta, d].X
                if value > 0.5:
                    area_employee_assignment.append([e, a, theta, d, shift])

area_employee_assignment_df = pd.DataFrame(area_employee_assignment, columns=['employee', 'area', 'period', 'day', 'shift'])
area_employee_assignment_df.head(10)

,employee,area,period,day,shift
0,0,10245,2,5,0
1,0,10245,3,5,0
2,0,10245,5,2,1
3,0,10245,5,6,1
4,0,10247,2,2,1
5,0,10247,2,3,1
6,0,10247,2,4,1
7,0,10247,2,6,1
8,0,10247,3,0,1
9,0,10247,3,6,1


In [1336]:
employees_shifts_df.query('employee == 0').sort_values(['day'])

,employee,shift,day,hours_worked
1,0,1,0,8
2,0,1,2,8
3,0,1,3,8
4,0,1,4,8
0,0,0,5,8
5,0,1,6,8


In [1337]:
(
    area_employee_assignment_df
    .query('employee == 0')
    .merge(couriers_needed_df, on=['day', 'area', 'period'])
    .assign(
        region = area_employee_assignment_df['area'].map(region_area_map)
    )
    .sort_values(['day', 'period'])
    [['employee', 'shift', 'day', 'period', 'area', 'region', 'n_couriers', 'deliveries']]
    .head(100)
)

,employee,shift,day,period,area,region,n_couriers,deliveries
19,0,1,0,2,10367,0,1.0,4.0
8,0,1,0,3,10247,0,1.0,4.0
10,0,1,0,4,10247,0,1.0,5.0
16,0,1,0,5,10318,0,1.0,5.0
4,0,1,2,2,10247,0,1.0,5.0
23,0,1,2,3,10369,0,1.0,5.0
18,0,1,2,4,10365,0,1.0,3.0
2,0,1,2,5,10245,0,1.0,5.0
5,0,1,3,2,10247,0,1.0,4.0
20,0,1,3,3,10367,0,1.0,4.0


In [1338]:
(
    area_employee_assignment_df
    .query('employee == 1')
    #.merge(couriers_needed_df, on=['day', 'area', 'period'])
    .assign(
        region = area_employee_assignment_df['area'].map(region_area_map)
    )
    .sort_values(['day', 'period'])
    [['employee', 'shift', 'day', 'period', 'area', 'region']] #, 'n_couriers']]
    .head(100)
)

,employee,shift,day,period,area,region
24,1,0,0,0,10245,0
34,1,0,0,1,10315,0
30,1,0,0,2,10249,0
45,1,0,0,3,10365,0
28,1,0,1,0,10249,0
35,1,0,1,1,10315,0
37,1,0,1,2,10317,0
32,1,0,1,3,10249,0
44,1,1,2,2,10365,0
43,1,1,2,3,10319,0


In [1328]:
# areas
area_period_days_df = (
    area_employee_assignment_df
    .groupby(['area', 'period', 'day'])
    .agg({'employee': ['count', 'unique']})
    .reset_index()
)
area_period_days_df.columns = ['area', 'period', 'day', 'employee_count', 'employees_assign']
area_period_days_df

,area,period,day,employee_count,employees_assign
0,10115,2,4,1,[3]
1,10115,2,6,1,[3]
2,10117,2,2,1,[3]
3,10117,2,3,1,[4]
4,10117,2,4,1,[4]
...,...,...,...,...,...
281,10999,3,5,1,[8]
282,10999,3,6,1,[7]
283,10999,4,2,1,[8]
284,10999,5,4,1,[7]


In [1329]:
# Outsourcing
outsourcing_shifts = []
for a in areas:
    for theta in Theta:
        for d in D:
            value = omega_var[area_map[a],theta,d].X
            #print(f'{a} {theta} {d} : {value}')
            if value > 0.0:
                outsourcing_shifts.append([a, theta, d, value])

outsourcing_shifts_df = pd.DataFrame(outsourcing_shifts, columns=['area', 'period', 'day', 'cost_outsource'])
outsourcing_shifts_df

,area,period,day,cost_outsource
0,10115,0,0,9.0
1,10115,0,1,6.0
2,10115,0,2,3.0
3,10115,0,3,9.0
4,10115,0,4,18.0
...,...,...,...,...
2458,10999,7,2,15.0
2459,10999,7,3,15.0
2460,10999,7,4,12.0
2461,10999,7,5,9.0


In [1330]:
# Join all
whole_solution_df = (
    couriers_needed_df
    # Employees
    .merge(area_period_days_df,  on=['area', 'period', 'day'], how='left')
    # Outsource
    .merge(outsourcing_shifts_df, on=['area', 'period', 'day'], how='left')
)
whole_solution_df.head(30)

,area,period,n_couriers,day,deliveries,employee_count,employees_assign,cost_outsource
0,10115,0,1.0,0,3.0,NaN,NaN,9.0
1,10117,0,0.0,0,0.0,NaN,NaN,NaN
2,10119,0,1.0,0,2.0,NaN,NaN,6.0
3,10178,0,1.0,0,1.0,NaN,NaN,3.0
4,10179,0,1.0,0,1.0,NaN,NaN,3.0
5,10243,0,1.0,0,5.0,1.0,[6],NaN
6,10245,0,1.0,0,5.0,1.0,[1],NaN
7,10247,0,1.0,0,3.0,NaN,NaN,9.0
8,10249,0,1.0,0,4.0,NaN,NaN,12.0
9,10315,0,1.0,0,2.0,NaN,NaN,6.0


In [1331]:
whole_solution_df.query('n_couriers == 2')

,area,period,n_couriers,day,deliveries,employee_count,employees_assign,cost_outsource
85,10559,1,2.0,0,7.0,NaN,NaN,21.0
115,10969,1,2.0,0,6.0,NaN,NaN,18.0
176,10999,2,2.0,0,7.0,NaN,NaN,21.0
182,10243,3,2.0,0,8.0,1.0,[7],12.0
236,10115,4,2.0,0,8.0,NaN,NaN,24.0
...,...,...,...,...,...,...,...,...
3193,10247,6,2.0,6,6.0,NaN,NaN,18.0
3196,10317,6,2.0,6,6.0,NaN,NaN,18.0
3204,10409,6,2.0,6,7.0,NaN,NaN,21.0
3207,10439,6,2.0,6,6.0,NaN,NaN,18.0


In [1332]:
whole_solution_df.query('n_couriers == 2 & employee_count == 2')

,area,period,n_couriers,day,deliveries,employee_count,employees_assign,cost_outsource
1142,10439,3,2.0,2,8.0,2.0,"[3, 4]",NaN
2558,10439,3,2.0,5,6.0,2.0,"[3, 5]",NaN


In [1333]:
theta = 2
area = 10999

a = area_map[area]
d = 0
r = region_area_map[area]
# couriers_needed[a, theta, d]
print(couriers_needed[a, theta, d])

# employees
for e in E[r]:
    val = k_var[e, a, theta, d].X
    if val > 0.5:
        print(e)

print(sum([k_var[e, a, theta, d].X for e in E[r]]))

# factor
factor = deliveries[a,theta,d] / couriers_needed[a,theta,d] * c_out
print(factor)

# outsource
print(omega_var[a, theta, d].X)

2.0
0.0
10.5
21.0
